In [1]:
import sys
import os
import io
import pathlib
import logging
import time

# Add to path the directory where tom_client.py exists
sys.path.insert( 0, str( pathlib.Path( os.getenv("HOME") ) / "research/desc/tom_desc" ) )
from tom_client import TomClient

# Get your DESC TOM username and password.  tompasswdfile
#   is a file with a single line containing your TOM password.
#   This should be in a place that's not readable by anybody
#   but you.  (Use this rather than putting the actual
#   password into something that might get committed to a
#   git archive!)
tomuser = 'rknop'
tompasswdfile = pathlib.Path( os.getenv("HOME") ) / "secrets/tom_rknop_passwd"

# TomClient is a thin front-end to Python requests that handles 
# authentication and some annoying headers that need to be set
# for connections to Django to work.
tomclient = TomClient( url="https://desc-tom.lbl.gov", username=tomuser, passwordfile=tompasswdfile )

# Make a logger so that we can print out timings and things like that
_logger = logging.getLogger("main")
if not _logger.hasHandlers():
    _logout = logging.StreamHandler( sys.stderr )
    _logger.addHandler( _logout )
    _logout.setFormatter( logging.Formatter( f'[%(asctime)s - %(levelname)s] - %(message)s',
                                             datefmt='%Y-%m-%d %H:%M:%S' ) )
_logger.setLevel( logging.INFO )
_logger.info( "Testing" )

[2024-10-24 13:27:54 - INFO] - Testing


In [2]:
# Make ourselves a convenience function that does some return value checking,
#   so we don't have to do that over and over again.

def query_tom( api_endpoint, data, verbose=False ):
    if verbose:
        _logger.info( f"Sending {api_endpoint} request to tom..." )
    t0 = time.perf_counter()
    res = tomclient.post( api_endpoint, json=data )
    dt = time.perf_counter() - t0
    if verbose:
        _logger.info( f"...done after {dt:.1f} seconds" )

    if res.status_code != 200:
        strio = io.StringIO()
        strio.write( f"Got status code {res.status_code}" )
        if verbose:
            strio.write( f"\n{res.text}" )
        raise RuntimeError( strio.getvalue() )

    retval = res.json()
    if isinstance(retval, dict) and ( 'status' in retval ) and ( retval['status'] != 'ok' ):
        strio = io.StringIO()
        strio.write( f"Got status {retval['status']}" )
        if verbose and ( 'error' in retval ): 
            strio.write( f"\n{retval['error']}" )
        raise RuntimeError (strio.getvalue() )

    return retval
    

In [4]:
# Ask for "hot" SNe.  This is the elasticc2/gethotsne API endpoint
# Documentation: https://github.com/LSSTDESC/tom_desc?tab=readme-ov-file#elasticc2hotsne

# Normally, this looks for supernovae whose latest detection
#   was some number of days before now.  This is what makes sense for a running
#   survey.  However, for a simulation, you want "now" to be different from the
#   actual time now.  So, there's a cheat parameter, mjd_now, that lets you tell
#   the server to pretend that the present time is different from the real
#   present time.
#
# The ELAsTiCC2 simulation has data from mjd 60796 to 61896, so pick
#   something randomly in the middle

retval = query_tom( "elasticc2/gethottransients", 
                   { "mjd_now": 61000., "detected_in_last_days": 1. },
                   verbose=True )

hotsne = retval['diaobject']

print( f"Got {len(hotsne)} hot transients!" )

[2024-10-24 13:31:44 - INFO] - Sending elasticc2/gethottransients request to tom...
[2024-10-24 13:32:38 - INFO] - ...done after 54.4 seconds


Got 54149 hot transients!


In [5]:
hotsne[0]

{'objectid': 1002611,
 'ra': 81.41968524696914,
 'dec': -49.968970477933674,
 'photometry': {'mjd': [60925.3889,
   60925.3985,
   60953.2899,
   60953.3125,
   60955.3221,
   60955.3225,
   60955.3474,
   60955.3479,
   60956.3486,
   60958.3617,
   60958.3728,
   60969.3496,
   60969.3616,
   60974.3396,
   60974.3515,
   60981.2149,
   60981.2398,
   60992.2305,
   60992.2547,
   60994.2755,
   60996.215,
   60996.2392,
   60999.3527],
  'band': ['Y',
   'Y',
   'Y',
   'Y',
   'i',
   'i',
   'z',
   'z',
   'i',
   'z',
   'Y',
   'i',
   'z',
   'r',
   'i',
   'z',
   'Y',
   'r',
   'i',
   'r',
   'g',
   'r',
   'z'],
  'flux': [-112.59107,
   -555.5123,
   5231.281,
   4139.281,
   6060.394,
   5658.994,
   5197.9297,
   5161.213,
   5684.1816,
   5602.6343,
   4597.311,
   5365.39,
   5534.702,
   4790.2104,
   4875.5986,
   4021.6533,
   4823.289,
   2758.784,
   3569.2063,
   2633.6055,
   619.1163,
   2358.463,
   3801.1348],
  'fluxerr': [1315.0219,
   1010.5355,
   933